In [173]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Beispielhafte Daten
# X = np.random.rand(100, 13)
# y = np.random.rand(100, 5)

# Deine echten Daten hier laden
X = ... # Dein Input-Datensatz (100, 13)
y = ... # Dein Output-Datensatz (100, 5)

In [174]:
import csv
import re

def read_csv_file(file_path):
    # Initialisiere eine leere Liste, um die bereinigten Daten zu speichern
    cleaned_data = []

    # Lese die CSV-Datei
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Annahme: Semikolon als Trennzeichen
        for row in reader:
            # Verbinde die Zeilenelemente mit einem Komma, um das Trennzeichen zu vereinheitlichen
            unified_row = ','.join(row)
            # Ersetze mehrere aufeinander folgende Kommas durch ein einzelnes Komma
            unified_row = re.sub(r',+', ',', unified_row)
            # Teile die vereinheitlichte Zeile nach dem Komma auf
            split_row = unified_row.split(',')
            # Entferne die ersten zwei Parameter
            cleaned_row = split_row[1:]
            # Füge die bereinigte Zeile der Liste hinzu
            cleaned_data.append(cleaned_row)

    return cleaned_data

# Beispiel: Daten aus "input.csv" einlesen

In [175]:
data = pd.read_csv("initial_data.csv")
cleaned_data = read_csv_file("querys_ForcePush.csv")
df_queried = pd.DataFrame(cleaned_data[1:], columns=cleaned_data[0])
def add_data(data, queried_data):
    # add queried data (without cost) to initial data
    print(queried_data.keys())
    queried_data = queried_data[queried_data['costs'] == 1]
    data = pd.concat([data, queried_data.iloc[:, :13]], axis=0)

    # data = data.append(queried_data, ignore_index=True)
    return data

data = add_data(data, df_queried)
data = data.reset_index().astype('Float32')

print(data)

#print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder']]

print(len(x))
print(len(y))

x_max = x.max()
x_min = x.min()
y_max = y.max()
y_min = y.min()
y_var = y.var()

#print(str(y_min) + "\n:\n" + str(y_max))

y_range = y_max - y_min


Index(['Engine speed', 'Engine load', 'Railpressure', 'Air supply',
       'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature',
       'NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder', 'costs'],
      dtype='object')
    index  Engine speed  Engine load  Railpressure   Air supply  Crank angle  \
0     0.0         700.0         7.33    500.899994    78.375298         1.91   
1     1.0         700.0        25.85    577.599976   118.067299         0.94   
2     2.0         700.0    46.669998    615.799988   144.355698         2.86   
3     3.0         700.0    69.550003    778.299988   174.177902         2.22   
4     4.0         700.0    94.330002    900.099976   213.314804        -1.72   
..    ...           ...          ...           ...          ...          ...   
95   95.0        2100.0   124.970001   2500.100098  1010.203125         4.27   
96   96.0        2200.0         12.6   2294.600098   321.264893          5.7   
97   97.0        2200.0    31.969999   240

C:\Users\boeke\AppData\Local\Temp\ipykernel_17072\3903568553.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, queried_data.iloc[:, :13]], axis=0)


In [176]:
# Datenaufteilung in Training und Test


print(len(x))
print(len(y))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)


# Datenstandardisierung
scaler_X = StandardScaler()
scaler_y = StandardScaler()

#X_train = scaler_X.fit_transform(X_train)
#X_test = scaler_X.transform(X_test)

#y_train = scaler_y.fit_transform(y_train)
#y_test = scaler_y.transform(y_test)


pls2 = PLSRegression(n_components=3)
pls2.fit(X_train, y_train)

test_results = pls2.predict(X_test)



100
100


In [177]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error


featurewise_mse = []
featurewise_map = []
print(test_results.shape)
print(y_test.shape)
y_test = np.transpose(y_test)
test_results = np.transpose(test_results)
for feature_t, feature_p in zip(test_results, y_test.values):
    # print(feature_t)
    # print(feature_p)
    # print(mean_squared_error(feature_t, feature_p))
    featurewise_map.append(mean_absolute_percentage_error(feature_t, feature_p))
    featurewise_mse.append(mean_squared_error(feature_t, feature_p))

featurewise_mse = np.array(featurewise_mse)

featurewise_map = np.array(featurewise_map)
#print(featurewise_mean)
# print(y_max.shape)
# print(featurewise_mean.shape)
# #print(y_range)
# featurewise_mean_percentage = featurewise_mean / y_max**2
# featurewise_mean_divided_by_variance = featurewise_mean / y_var
print("mean_squared_error:")
for mean in featurewise_mse:
    print('{:<10f}'.format(mean))
print("mean_absolute_percentage_error:")
for mean in featurewise_map:
    print('{:<10f}'.format(mean*100))

# print("mse per feature in percentage to value range")
# print(featurewise_mean_percentage)
# print("featurewise_mean_divided_by_variance:")
# print(featurewise_mean_divided_by_variance)

(10, 5)
(10, 5)
mean_squared_error:
2946.870414
1.696068  
29.100488 
25.930863 
18.317407 
mean_absolute_percentage_error:
72.632742 
78.835890 
58.856016 
77.323406 
5.158597  


In [178]:

submission_data = pd.read_csv("submission.csv")


#print(submission_data)
#print(submission_data.keys())

submission_keys = ['Speed', 'Load', 'Railpressure', 'Air', 'Angle', 'Pressure1','Pressure2', 'Temperature']
model_keys = ['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']

for sub_key, mod_key in zip(submission_keys, model_keys):
    submission_data = submission_data.rename(columns={sub_key:mod_key})



# input = input.reshape((1,8))
submission_result = pls2.predict(submission_data)


submission_result = np.transpose(submission_result)
submission_std = []
submission_mean = []
#print(submission_result)
for feature in submission_result:
    submission_std.append(feature.std())
    submission_mean.append(feature.mean())

print(submission_mean)
print(submission_std)

y_std = []
y_mean = []
#print(y)
y_t = np.transpose(np.array(y))

for feature in y_t:
    #print(feature)
    y_std.append(feature.std())
    y_mean.append(feature.mean())


print(y_mean)
print(y_std)

q_mean = np.divide(y_mean, submission_mean)
q_std = np.divide(y_std, submission_std)

print("\nQuotienten von mean und std:\n")
print(q_mean)
print(q_std)

# print(result)

[263.1563737387438, 2.2356318186765023, 83.79336361410725, 4.709627978763616, 106.5400573624858]
[88.87191484400569, 0.7323163829270714, 25.857270103130983, 1.69463046712027, 21.237226518494598]
[170.29038274765014, 1.9017370684444905, 59.82909906625748, 5.4507354098558425, 82.80317947387695]
[133.46353806052107, 1.29944648318814, 40.135816557622924, 3.0441017736734306, 28.307689588618114]

Quotienten von mean und std:

[0.64710719 0.8506486  0.71400761 1.15736008 0.77720232]
[1.50175157 1.77443317 1.55220626 1.79632187 1.3329278 ]
